## Reading/Processing files of a trained dataset.

In [1]:
# Basic imports and paths
from plyfile import PlyData, PlyElement
import json
import torch
from gs_lib import quaternion_to_rot, strip_symmetric, focal2fov, fov2focal, getWorld2View, getProjectionMatrix, scale_res, get_padded_size


# The only two files that are needed for rendering.
dataset = f"/home/suresh/2024/gs/datasets/models/truck"
iteration = "30000"
max_sh_degree = 3
json_file = f"{dataset}/cameras.json"
ply_file = f"{dataset}/point_cloud/iteration_{iteration}/point_cloud.ply"

# Print to see if proper
print(f"JSON = {json_file}")
print(f"PLY = {ply_file}")

# Check for the available paraller device
device = "mps" if getattr(torch,'has_mps',False) else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device running on = {device}")

JSON = /home/suresh/2024/gs/datasets/models/truck/cameras.json
PLY = /home/suresh/2024/gs/datasets/models/truck/point_cloud/iteration_30000/point_cloud.ply
Device running on = cuda


## PLY File stuff

In [2]:
# Readind the ply file.
plydata = PlyData.read(ply_file)
print("The structure of the ply file:")
print("------------------------------")
print(plydata.elements[0])

The structure of the ply file:
------------------------------
element vertex 2541226
property float x
property float y
property float z
property float nx
property float ny
property float nz
property float f_dc_0
property float f_dc_1
property float f_dc_2
property float f_rest_0
property float f_rest_1
property float f_rest_2
property float f_rest_3
property float f_rest_4
property float f_rest_5
property float f_rest_6
property float f_rest_7
property float f_rest_8
property float f_rest_9
property float f_rest_10
property float f_rest_11
property float f_rest_12
property float f_rest_13
property float f_rest_14
property float f_rest_15
property float f_rest_16
property float f_rest_17
property float f_rest_18
property float f_rest_19
property float f_rest_20
property float f_rest_21
property float f_rest_22
property float f_rest_23
property float f_rest_24
property float f_rest_25
property float f_rest_26
property float f_rest_27
property float f_rest_28
property float f_rest_29
prop

In [3]:
# extract the XYZ coordinate of gaussians
print(type(plydata.elements[0]["x"]))
x = torch.tensor(plydata.elements[0]["x"])
y = torch.tensor(plydata.elements[0]["y"])
z = torch.tensor(plydata.elements[0]["z"])
xyz = torch.stack((x,y,z), dim=1)
print(f"XYZ = {xyz.shape}, e.g. xyz[0] = {xyz[0]}")

<class 'numpy.memmap'>
XYZ = torch.Size([2541226, 3]), e.g. xyz[0] = tensor([-0.1773,  1.4565, -0.6173])


In [4]:
# extract the opacity of gaussians
op = torch.tensor(plydata.elements[0]["opacity"])
print(f"op = {op.shape}")


op = torch.Size([2541226])


In [5]:
# extract spherical harmonics, DC component of gaussians
sh_dc = torch.zeros((xyz.shape[0], 3, 1))
sh_dc[:, 0, 0] = torch.tensor(plydata.elements[0]["f_dc_0"])
sh_dc[:, 1, 0] = torch.tensor(plydata.elements[0]["f_dc_1"])
sh_dc[:, 2, 0] = torch.tensor(plydata.elements[0]["f_dc_2"])
print(f"sh_dc = {sh_dc.shape}")
print(f"sh_dc = {sh_dc[0]}")

sh_dc = torch.Size([2541226, 3, 1])
sh_dc = tensor([[-0.3723],
        [-0.4185],
        [-0.2502]])


In [6]:
# These are all the properties of a single vertex available in the ply file
print(f"{plydata.elements[0].properties}")

(PlyProperty('x', 'float'), PlyProperty('y', 'float'), PlyProperty('z', 'float'), PlyProperty('nx', 'float'), PlyProperty('ny', 'float'), PlyProperty('nz', 'float'), PlyProperty('f_dc_0', 'float'), PlyProperty('f_dc_1', 'float'), PlyProperty('f_dc_2', 'float'), PlyProperty('f_rest_0', 'float'), PlyProperty('f_rest_1', 'float'), PlyProperty('f_rest_2', 'float'), PlyProperty('f_rest_3', 'float'), PlyProperty('f_rest_4', 'float'), PlyProperty('f_rest_5', 'float'), PlyProperty('f_rest_6', 'float'), PlyProperty('f_rest_7', 'float'), PlyProperty('f_rest_8', 'float'), PlyProperty('f_rest_9', 'float'), PlyProperty('f_rest_10', 'float'), PlyProperty('f_rest_11', 'float'), PlyProperty('f_rest_12', 'float'), PlyProperty('f_rest_13', 'float'), PlyProperty('f_rest_14', 'float'), PlyProperty('f_rest_15', 'float'), PlyProperty('f_rest_16', 'float'), PlyProperty('f_rest_17', 'float'), PlyProperty('f_rest_18', 'float'), PlyProperty('f_rest_19', 'float'), PlyProperty('f_rest_20', 'float'), PlyProperty('

In [7]:
# extract rest of the SH components of gaussians
# First we exract all the available SH componenets that are under the name "f_rest_<n>"
# We extract the string of the property and sort it. Then we extract the floating values of those properties
# into a tensor. We can take only the amount we need. 
# Maximum of 3 degree SH is used in all implementations and training
# 3degree SH = 3+3*15 components per degree, hence 3(f_dc_0 to f_dc_2) + 45(f_rest_0 to f_rest_44)

extra_f_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("f_rest_")]
extra_f_names = sorted(extra_f_names, key = lambda x: int(x.split('_')[-1]))
assert len(extra_f_names) == 3*(max_sh_degree + 1) ** 2 - 3

sh_extra = torch.zeros((xyz.shape[0], len(extra_f_names)))
for idx, attr_name in enumerate(extra_f_names):
    sh_extra[:, idx] = torch.tensor(plydata.elements[0][attr_name])
sh_extra = sh_extra.reshape((sh_extra.shape[0], 3, (max_sh_degree + 1) ** 2 - 1))

print(f"sh_extra = {sh_extra.shape}")

sh_extra = torch.Size([2541226, 3, 15])


In [8]:
# extract the scales and rotations of gaussians
# Similar to SH_rest, this is also done.
scale_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("scale_")]
scale_names = sorted(scale_names, key = lambda x: int(x.split('_')[-1]))
scales = torch.zeros((xyz.shape[0], len(scale_names)))
for idx, attr_name in enumerate(scale_names):
    scales[:, idx] = torch.tensor(plydata.elements[0][attr_name])

rot_names = [p.name for p in plydata.elements[0].properties if p.name.startswith("rot")]
rot_names = sorted(rot_names, key = lambda x: int(x.split('_')[-1]))
rots = torch.zeros((xyz.shape[0], len(rot_names)))
for idx, attr_name in enumerate(rot_names):
    rots[:, idx] = torch.tensor(plydata.elements[0][attr_name])
    
print(f"Scales = {scales.shape}")
print(f"Rotations = {rots.shape}")    

Scales = torch.Size([2541226, 3])
Rotations = torch.Size([2541226, 4])


In [9]:
# The scales, rotations and sh components are processed by some activation.
# Right now, not really sure why. Probably has to do with backpropagation during training using pytorch


# Scales are passed thru exp()
# rotations are normalized thru torch.nn.functional.normalize()
# opacities are passed thru sigmoid
#
# https://github.com/graphdeco-inria/gaussian-splatting/blob/d9fad7b3450bf4bd29316315032d57157e23a515/scene/gaussian_model.py#L33
scales = torch.exp(scales)
rots = torch.nn.functional.normalize(rots)
op = torch.sigmoid(op)

In [10]:
# Building a single tensor for all SH features
sh_all = torch.cat([sh_dc, sh_extra], dim=2)
print(f"SH components = {sh_all.shape}")

SH components = torch.Size([2541226, 3, 16])


In [11]:
# Now we build all the other required components for GS rendering.
# Refer to the gs_lib.py file for more info of the functions used. 
# They are defined in the order of appreance in the notebook.

# First we need calculate the 3D covariance sigma(E).
# This is the equation 6 in the official paper
# E = R * S * S^T * R^T
# "*" measn matrixmult, "^T" means transpose
# S = Scaling matrix, R = Rotation Matrix

# First lets build the rotation matrix using quaternions
# https://www.songho.ca/math/quaternion/quaternion.html
# The above link gives a great derivation
# https://www.songho.ca/opengl/gl_quaternion.html - This formula is used in GS
rots = quaternion_to_rot(rots)

# Convert 3-values array to diagonal vector for scale matrix
Scales = torch.zeros((scales.shape[0], 3, 3), dtype=torch.float32)
Scales[:, 0, 0] = scales[:, 0]
Scales[:, 1, 1] = scales[:, 1]
Scales[:, 2, 2] = scales[:, 2]

# now calculate sigma  E = R * S * S^T * R^T
E_raw = rots @ Scales
E_raw = E_raw @ E_raw.transpose(2, 1)


# Now E is our covariance matrix.
# Covariance matrices are symmetrical across the diagonal.
# Hence, I think thats why the official paper strips the bottom 
# elements from the diagonal.
# https://github.com/graphdeco-inria/gaussian-splatting/blob/d9fad7b3450bf4bd29316315032d57157e23a515/utils/general_utils.py#L64
E = strip_symmetric(E_raw)

In [12]:
del Scales, rot_names, scale_names, scales, rots

## JSON File stuffs

In [13]:
# Each image used for training has its own parameters.
# We can use the Camera intrinsic and extrinsic parameters used by the training images to render
# In other words, lets render a image of the same size and resolution as the training
with open(json_file) as f:
    json_raw = f.read()
json_parsed = json.loads(json_raw)
print(f"Images = {len(json_parsed)}")

img = 52  # choose a random image
camera = json_parsed[img]  # has height, width, position, rotation, focal_x, focal_y
# print(f"{camera}")
w = camera['width']  # one value
h = camera['height'] # one value
focal_x = camera['fx']  # one value
focal_y = camera['fy']  # one value
cam_rotation = torch.tensor(camera['rotation'])  # 3x3 matrix
cam_position = torch.tensor(camera['position'])  # 3-value vector

fov_x = focal2fov(focal_x, w)
fov_y = focal2fov(focal_y, h)

# Set a z axis(depth) boundry. So gaussian before and after these bounds will be removed.
# These are called clipping planes
znear = 0.1
zfar = 100.0 


# Scaling down and block adjusting image sizes
# We are doing this so that our output image size is divisible equally by the block size.
# Also only focal lenght is dependent on the final image size. FOV remains the same.
new_w, new_h = scale_res(w, h, 2)
TILE_SIZE  = 16
final_w, final_h, total_blocks = get_padded_size(new_w, new_h, TILE_SIZE)
focal_x = fov2focal(fov_x, new_w)
focal_y = fov2focal(fov_y, new_h)

print(f"Tile size \t\t\t= {TILE_SIZE}")
print(f"Total blocks \t\t\t= {total_blocks}")
print(f"Original Width x Height \t= {w} x {h}")
print(f"Block-adjusted Width x Height \t= {final_w} x {final_h}")
print(f"Focal_x \t\t\t= {focal_x}")
print(f"Focal_y \t\t\t= {focal_y}")
print(f"FOV_x \t\t\t\t= {fov_x}")
print(f"FOV_y \t\t\t\t= {fov_y}")

Images = 251
Tile size 			= 16
Total blocks 			= 2170
Original Width x Height 	= 1957 x 1091
Block-adjusted Width x Height 	= 992 x 560
Focal_x 			= 581.3302001953125
Focal_y 			= 578.6701049804688
FOV_x 				= 1.3986958265304565
FOV_y 				= 0.8816215991973877


In [14]:
cam_translation = cam_position @ torch.linalg.inv(-cam_rotation.T)
view_transformation_matrix = getWorld2View(cam_rotation, cam_translation).T
projection_matrix = getProjectionMatrix(znear, zfar, fov_x, fov_y).T
full_transformation_matrix = view_transformation_matrix @ projection_matrix
print(f"Full projection matrix : {full_transformation_matrix}")

Full projection matrix : tensor([[ 0.9527, -0.5068,  0.5489,  0.5483],
        [ 0.2265,  2.0580,  0.1450,  0.1449],
        [-0.6741, -0.0246,  0.8244,  0.8236],
        [-1.9953, -0.6074,  1.4858,  1.5843]])
